# Bidirectional Vanilla RNN corpus Otomi

In [1]:
from reccurrent_model import biRNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np
import torch

In [2]:
file1 = load(open('pickle_objects/preinput_data','rb'))
print(file1[0], len(file1))

[[["'", 'v', 'B-como'], ['á', 'v', 'I-como'], ['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-ctrf'], ['á', 'v', 'I-ctrf'], ['k', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['á', 'v', 'I-stem']]] 1769


In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.1) #33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_sents, train_tags = zip(*train_pairs)

1592 177


In [5]:
model = biRNN(train_sents, train_tags)

In [6]:
%%time
model.train_model(its=150)

100%|██████████| 150/150 [37:31<00:00, 15.01s/it]

CPU times: user 6h 42min 36s, sys: 1min 12s, total: 6h 43min 48s
Wall time: 37min 31s


### Evaluation

In [7]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = model.forward(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [8]:
y_true = list(chain(*y_true))
y_pred = list(chain(*y_pred))

In [9]:
labels = list(set(y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(y_true, y_pred))

Accuracy: 0.7421742808798646


In [10]:
prec, rec, f1, supp = precision_recall_fscore_support(y_true, y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
I-dist,0.000000,0.0000,0.000000,1.0
I-mov,0.000000,0.0000,0.000000,1.0
B-solo,0.000000,0.0000,0.000000,2.0
B-dem,0.000000,0.0000,0.000000,30.0
B-det,0.869565,0.9375,0.902256,64.0
...,...,...,...,...
B-dist,0.000000,0.0000,0.000000,1.0
I-dios,0.000000,0.0000,0.000000,2.0
B-lig,0.000000,0.0000,0.000000,28.0
I-pues,0.000000,0.0000,0.000000,2.0


In [12]:
print(results.to_string())

             Precision    Recall        F1  Support
I-dist        0.000000  0.000000  0.000000      1.0
I-mov         0.000000  0.000000  0.000000      1.0
B-solo        0.000000  0.000000  0.000000      2.0
B-dem         0.000000  0.000000  0.000000     30.0
B-det         0.869565  0.937500  0.902256     64.0
I-lim         0.000000  0.000000  0.000000     71.0
I-1.pss       0.000000  0.000000  0.000000     10.0
I-1.obj       0.000000  0.000000  0.000000     10.0
B-2.cpl       0.000000  0.000000  0.000000      1.0
B-cuando      0.000000  0.000000  0.000000      1.0
B-det.pl      0.000000  0.000000  0.000000     24.0
I-prag        0.000000  0.000000  0.000000     46.0
I-solo        0.000000  0.000000  0.000000      4.0
I-1.cnt       0.279070  1.000000  0.436364     12.0
B-1.pss       0.000000  0.000000  0.000000     10.0
I-aqui        0.000000  0.000000  0.000000      2.0
B-pues        0.000000  0.000000  0.000000      2.0
I-2.cpl       0.000000  0.000000  0.000000      1.0
I-aum       

In [13]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.5019175698571023
Average recall: 0.7015911872705018
Average F1: 0.5839367852912165


In [14]:
#torch.save(model,'Model.biLSTM.justChar')

#### Save embeddings

In [15]:
embs = {}
for w,v in model.input_voc.items():
    if w != ' ':
        vec = model.emb[0](torch.tensor([v])).detach().numpy()[0]
        embs[w] = vec   

In [16]:
from pickle import dump

dump( embs, open( "Embs.biRNN.justChar.p", "wb" ) )